In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00


In [2]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires to

# Example of Future scrapped tweets

In [3]:
scrapped_tweets = [
"""Men think women are lying when we tell our chubby or less attractive girl friend that she’s beautiful.

In reality we just have a less shallow view of beauty, which men should be grateful for since most of them aren’t all that attractive either.""",
"""
Number of Palestinian refugees being  taken in by Arab countries:  0

Number of Ukrainian refugees taken in by European countries: 6 million

Stop pretending it’s totally normal that  none of the surrounding Arab countries  will take these people or that it’s Israel alone “trapping them there”. The entire Muslim world is trapping them there.""",
"""
Colorado shooter is non-binary, according to attorney. “Could this be a strategy to get out of hate crime charges?”

The only requirement for being trans is saying the magic words . When women point out this obvious loophole we’re “bigots”.

Enjoy the monster you’ve created.
""",
 ]

# Example of future labels scores

In [12]:
labels = ['bullying', "insult", "profanity", 'sarcasm', 'threat', 'exclusion', 'porn', 'spam', 'lgbt', 'black lives matter', 'ecological activism', "feminism", 'politics', 'positive', 'negative', ]

# All code sample

## At first step we need to label our tweets for future generation. We use **facebook/bart-large-mnli** model by default

In [5]:
import pandas as pd
from transformers import pipeline

classifier = pipeline("zero-shot-classification")


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
classification_results = []

for tweet in scrapped_tweets:
    result = classifier(tweet, labels)
    classification_results.append({
        'tweet': result['sequence'],
        'labels': result['labels'],
        'scores': result['scores']
    })

In [8]:

df = pd.DataFrame(classification_results)

# Future DataSet will looks like this:

In [9]:
df

,tweet,labels,scores
0,Men think women are lying when we tell our chu...,"[threat, exclusion, negative, sarcasm, positiv...","[0.24295514822006226, 0.16632391512393951, 0.1..."
1,\nNumber of Palestinian refugees being taken ...,"[negative, profanity, exclusion, threat, sarca...","[0.23413468897342682, 0.22197233140468597, 0.2..."
2,"\nColorado shooter is non-binary, according to...","[negative, threat, exclusion, bullying, insult...","[0.5738505721092224, 0.12189709395170212, 0.08..."


In [13]:
df_2 = pd.DataFrame(columns=['tweet'] + labels)

for tweet in scrapped_tweets:
    result = classifier(tweet, labels)

    result_dict = {label: score for label, score in zip(result['labels'], result['scores'])}
    print("Tweet - ", tweet)
    print(result_dict)
    result_dict['tweet'] = tweet

    df_2 = df_2.append(result_dict, ignore_index=True)


Tweet -  Men think women are lying when we tell our chubby or less attractive girl friend that she’s beautiful. 

In reality we just have a less shallow view of beauty, which men should be grateful for since most of them aren’t all that attractive either.
{'threat': 0.24090297520160675, 'exclusion': 0.16491901874542236, 'negative': 0.15995849668979645, 'sarcasm': 0.13789932429790497, 'positive': 0.07281239330768585, 'lgbt': 0.04462684318423271, 'insult': 0.043057117611169815, 'profanity': 0.03586936742067337, 'spam': 0.029417436569929123, 'black lives matter': 0.01831200160086155, 'bullying': 0.013748841360211372, 'ecological activism': 0.013520675711333752, 'feminism': 0.008446731604635715, 'politics': 0.008300627581775188, 'porn': 0.008208172395825386}


<ipython-input-13-1cc8b3c975fb>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2 = df_2.append(result_dict, ignore_index=True)


Tweet -  
Number of Palestinian refugees being  taken in by Arab countries:  0

Number of Ukrainian refugees taken in by European countries: 6 million

Stop pretending it’s totally normal that  none of the surrounding Arab countries  will take these people or that it’s Israel alone “trapping them there”. The entire Muslim world is trapping them there.
{'negative': 0.23315821588039398, 'profanity': 0.2210465669631958, 'exclusion': 0.20217818021774292, 'threat': 0.1018519327044487, 'sarcasm': 0.07638708502054214, 'insult': 0.06958746165037155, 'bullying': 0.0200132355093956, 'lgbt': 0.018798548728227615, 'spam': 0.012427452020347118, 'politics': 0.011379719711840153, 'ecological activism': 0.008827139623463154, 'positive': 0.007696752902120352, 'black lives matter': 0.006476311478763819, 'porn': 0.006000894121825695, 'feminism': 0.004170597996562719}


<ipython-input-13-1cc8b3c975fb>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2 = df_2.append(result_dict, ignore_index=True)


Tweet -  
Colorado shooter is non-binary, according to attorney. “Could this be a strategy to get out of hate crime charges?”

The only requirement for being trans is saying the magic words . When women point out this obvious loophole we’re “bigots”. 

Enjoy the monster you’ve created.

{'negative': 0.5723560452461243, 'threat': 0.12157963216304779, 'exclusion': 0.08626818656921387, 'bullying': 0.06734674423933029, 'insult': 0.0632706880569458, 'sarcasm': 0.03317606821656227, 'profanity': 0.016147544607520103, 'lgbt': 0.0141491973772645, 'spam': 0.0064994036220014095, 'politics': 0.005867647938430309, 'positive': 0.00419601472094655, 'porn': 0.002956849755719304, 'feminism': 0.0026044107507914305, 'black lives matter': 0.002406557323411107, 'ecological activism': 0.0011751115089282393}


<ipython-input-13-1cc8b3c975fb>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2 = df_2.append(result_dict, ignore_index=True)


# Or like this

In [14]:
df_2

,tweet,bullying,insult,profanity,sarcasm,threat,exclusion,porn,spam,lgbt,black lives matter,ecological activism,feminism,politics,positive,negative
0,Men think women are lying when we tell our chu...,0.013749,0.043057,0.035869,0.137899,0.240903,0.164919,0.008208,0.029417,0.044627,0.018312,0.013521,0.008447,0.008301,0.072812,0.159958
1,\nNumber of Palestinian refugees being taken ...,0.020013,0.069587,0.221047,0.076387,0.101852,0.202178,0.006001,0.012427,0.018799,0.006476,0.008827,0.004171,0.011380,0.007697,0.233158
2,"\nColorado shooter is non-binary, according to...",0.067347,0.063271,0.016148,0.033176,0.121580,0.086268,0.002957,0.006499,0.014149,0.002407,0.001175,0.002604,0.005868,0.004196,0.572356


In [15]:
from transformers import AutoTokenizer

model_name='google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def count_tokens(text):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    num_tokens = input_ids.shape[1]
    return num_tokens

token_counts = df_2['tweet'].apply(count_tokens)

average_token_count = token_counts.mean()
print(f"Number of tokens: {average_token_count}")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Number of tokens: 65.0


# Code for creating prompt

In [16]:
import pandas as pd

def creating_json(index: int, dataset: pd.DataFrame) -> dict:
    """
    The function creates a dictionary of all label scores from the dataset for a given index.

    :param index: Index of the row in the dataset.
    :param dataset: DataFrame containing tweets and label scores.
    :return: Dictionary with labels and their corresponding scores.
    """
    if index < 0 or index >= len(dataset):
        raise IndexError("Index is out of bounds for the dataset.")

    row = dataset.iloc[index]

    label_scores = {label: row[label] for label in dataset.columns if label != 'tweet'}

    return label_scores

In [17]:
index = 0

try:
    index_to_check = 0  # The index of the row we want to check
    label_scores_dict = creating_json(index_to_check, df_2)
    print(label_scores_dict)
except IndexError as e:
    print(e)


{'bullying': 0.013748841360211372, 'insult': 0.043057117611169815, 'profanity': 0.03586936742067337, 'sarcasm': 0.13789932429790497, 'threat': 0.24090297520160675, 'exclusion': 0.16491901874542236, 'porn': 0.008208172395825386, 'spam': 0.029417436569929123, 'lgbt': 0.04462684318423271, 'black lives matter': 0.01831200160086155, 'ecological activism': 0.013520675711333752, 'feminism': 0.008446731604635715, 'politics': 0.008300627581775188, 'positive': 0.07281239330768585, 'negative': 0.15995849668979645}


# Example of one-shot learning on flan-t5

In [18]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [22]:
index = 0

labels = creating_json(index, df_2)
tweet = df_2['tweet'][index]

# prompt = f"""
# Create a tweet based on the presence of the following features(1 means the explicit presence of this feature, 0 means the complete absence of this feature).

# {labels}

# Tweet:
# """

prompt = f"""
Create a tweet that reflects the proportional representation of the following features. Each feature's presence in the tweet should correspond to its percentage in the overall probability distribution (where the sum of all feature probabilities equals 1). Please ensure that the content and tone of the tweet are influenced accordingly by these proportions.

Feature Probability Distribution:
{labels}

Tweet:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=average_token_count+5,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN TWEET:\n{tweet}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
  
Create a tweet that reflects the proportional representation of the following features. Each feature's presence in the tweet should correspond to its percentage in the overall probability distribution (where the sum of all feature probabilities equals 1). Please ensure that the content and tone of the tweet are influenced accordingly by these proportions.  
  
Feature Probability Distribution:  
{'bullying': 0.013748841360211372, 'insult': 0.043057117611169815, 'profanity': 0.03586936742067337, 'sarcasm': 0.13789932429790497, 'threat': 0.24090297520160675, 'exclusion': 0.16491901874542236, 'porn': 0.008208172395825386, 'spam': 0.029417436569929123, 'lgbt': 0.04462684318423271, 'black lives matter': 0.01831200160086155, 'ecological activism': 0.013520675711333752, 'feminism': 0.008446731604635715, 'politics': 0.008300627581775188, 'positive': 0.07281239330768585, 'negativ